##### Copyright 2020 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Point Clouds for tensorflow_graphics
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/schellmi42/tensorflow_graphics_point_clouds/blob/master/pylib/notebooks/ModelNet40.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/schellmi42/tensorflow_graphics_point_clouds/blob/master/pylib/notebooks/ModelNet40.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Initialization


### Clone repositories, install requirements and custom_op package

In [ ]:
# Clone repositories
!rm -r graphics
!git clone https://github.com/schellmi42/graphics

# install requirements and load tfg module 
!pip install -r graphics/requirements.txt

# install custom ops
!pip install graphics/tensorflow_graphics/projects/point_convolutions/custom_ops/pkg_builds/tf_2.2.0/*.whl


### Load modules

In [ ]:
import sys
# (this is equivalent to export PYTHONPATH='$HOME/graphics:/content/graphics:$PYTHONPATH', but adds path to running session)
sys.path.append("/content/graphics")

# load point cloud module 
# (this is equivalent to export PYTHONPATH='/content/graphics/tensorflow_graphics/projects/point_convolutions:$PYTHONPATH', but adds path to running session)
sys.path.append("/content/graphics/tensorflow_graphics/projects/point_convolutions")

Check if it loads without errors

In [ ]:
import tensorflow as tf
import tensorflow_graphics as tfg
import pylib.pc as pc
import numpy as np

print('TensorFlow version: %s'%tf.__version__)
print('TensorFlow-Graphics version: %s'%tfg.__version__)
print('Point Cloud Module: ', pc)

## Classification on ModelNet40

### Data preparation

First we load the data consisting of 10k points per model.

In [ ]:
!wget --no-check-certificate https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip 
!echo '---unzipping---'
!unzip -q modelnet40_normal_resampled.zip 
!echo '[done]'

Next we load the data, using the input function in the `io` module.

To speed up this tutorial, we only load a subset of the points per model.



In [ ]:
import tensorflow as tf 
import pylib.pc as pc
import pylib.io as io
import numpy as np
import tensorflow_graphics
import os, time


quick_test = False  # only load 100 models

# -- loading data ---

data_dir = 'modelnet40_normal_resampled/'
num_classes = 40  # modelnet 10 or 40
points_per_file = 5000  # number of points loaded per model

# load category names
category_names = []
with open(data_dir + f'modelnet{num_classes}_shape_names.txt') as inFile:
  for line in inFile:
    category_names.append(line.replace('\n', ''))

# load names of training files
train_set = []
train_labels = []
with open(data_dir + f'modelnet{num_classes}_train.txt') as inFile:
  for line in inFile:
    line = line.replace('\n', '')
    category = line[:-5]
    train_set.append(data_dir + category + '/' + line + '.txt')
    if category not in category_names:
      raise ValueError('Unknown category ' + category)
    train_labels.append(category_names.index(category))

# load names of test files
test_set = []
test_labels = []
with open(data_dir + f'modelnet{num_classes}_test.txt') as inFile:
  for line in inFile:
    line = line.replace('\n', '')
    category = line[:-5]
    test_set.append(data_dir + category + '/' + line + '.txt')
    if category not in category_names:
      raise ValueError('Unknown category ' + category)
    test_labels.append(category_names.index(category))

# load training data
train_data_points = np.empty([len(train_set), points_per_file, 3])

print(f'### loading modelnet{num_classes} train ###')
for i, filename in enumerate(train_set):
  points, _ = \
      io.load_points_from_file_to_numpy(filename,
                                        max_num_points=points_per_file)
  train_data_points[i] = points
  if i % 500 == 0:
    print(f'{i}/{len(train_set)}')
  if quick_test and i > 100:
    break

# load test data
test_data_points = np.empty([len(test_set), points_per_file, 3])

print(f'### loading modelnet{num_classes} test ###')
for i, filename in enumerate(test_set):
  points, _ = \
      io.load_points_from_file_to_numpy(filename,
                                        max_num_points=points_per_file)
  test_data_points[i] = points
  if i % 500 == 0:
    print(f'{i}/{len(test_set)}')
  if quick_test and i > 100:
    break

Now let's define a small data loader.

To make the network evaluation faster, we randomly samples the point clouds to reduce the input size.

As we don't want to provide any additional features other than the point location to the network, we will use a constant `1` as input feature.





In [ ]:
class modelnet_data_generator(tf.keras.utils.Sequence):
  ''' Small generator of batched data.
  '''
  def __init__(self,
               points,
               labels,
               batch_size):
      self.points = points
      self.labels = np.array(labels, dtype=int)
      self.batch_size = batch_size
      self.epoch_size = len(self.points)

      self.ids = np.arange(0, points_per_file)
      # shuffle data before training
      self.on_epoch_end()

  def __len__(self):
    # number of batches per epoch
    return(int(np.floor(self.epoch_size / self.batch_size)))

  def __call__(self):
    ''' Loads batch and increases batch index.
    '''
    data = self.__getitem__(self.index)
    self.index += 1
    return data

  def __getitem__(self, index, samples_per_model=1024):
    ''' Loads data of current batch and samples random subset of the points.
    '''
    labels = \
        self.labels[index * self.batch_size:(index + 1) * self.batch_size]
    points = \
        self.points[index * self.batch_size:(index + 1) * self.batch_size]
    # constant input feature
    features = tf.ones([self.batch_size, samples_per_model, 1])

    # sample points
    sampled_points = np.empty([self.batch_size, samples_per_model, 3])
    for batch in range(self.batch_size):
      selection = np.random.choice(self.ids, samples_per_model)
      sampled_points[batch] = points[batch][selection]

    return sampled_points, features, labels

  def on_epoch_end(self):
    ''' Shuffles data and resets batch index.
    '''
    shuffle = np.random.permutation(np.arange(0, len(self.points)))
    self.points = self.points[shuffle]
    self.labels = self.labels[shuffle]
    self.index = 0

### Network architecture

Let's build a simple classification network, which uses point convolutions for encoding the shape, and two dense layers for predicting the class.

The following model contains example calls for the three different available point convolutions in the `layer` module:


*   [Monte-Carlo convolutions](https://www.uni-ulm.de/fileadmin/website_uni_ulm/iui.inst.100/institut/Papers/viscom/2018/hermosilla2018montecarlo.pdf), which uses MLPs for representing the convolutional kernel, and aggregates the features inside the convolution radius using [Monte-Carlo integration](https://en.wikipedia.org/wiki/Monte_Carlo_integration), where each feature is weighted by a point density estimation.
*   [Kernel Point convolutions](https://arxiv.org/pdf/1904.08889.pdf), where the convolutional kernel is represented by a set of weights on kernel points, which are interpolated. 
(Note: We use rigid kernel points in the example below but deformable kernel points are also supported)

*   [PointConv convolutions](https://openaccess.thecvf.com/content_CVPR_2019/papers/Wu_PointConv_Deep_Convolutional_Networks_on_3D_Point_Clouds_CVPR_2019_paper.pdf)
, which uses a single MLP for representing the convolutional kernel, and aggregates the features using an integration, where each feature is weighted by a learned inverse density estimation.


Note that different to an image convolution layer, a point convolution layer needs additional input about the spatial location of the features, i.e. point coordinates.
In case of a 'strided' point convolution, where the output features are defined on different points than the input, we have to provide two point clouds.

For sampling the point clouds to lower densities, we use the `PointHierarchy` class.

At the end of the encoder we use a `GlobalAveragePooling` layer to aggregate the features from all points into one latent vector. 

In [ ]:
from pylib.pc import layers

class mymodel(tf.keras.Model):
  ''' Model architecture with `L` convolutional layers followed by 
  two dense layers.

  Args:
    features_sizes: A `list` of `ints`, the feature dimensions. Shape `[L+3]`.
    sample_radii: A `list` of `floats, the radii used for sampling
      of the point clouds. Shape `[L]`.
    conv_radii: A `list` of `floats`, the radii used by the convolution
      layers. Shape `[L]`.
    layer_type: A `string`, the type of convolution used,
      can be 'MCConv', 'KPConv', 'PointConv'.
    sampling_method: 'poisson disk' or 'cell average'.
  '''

  def __init__(self,
               feature_sizes,
               sample_radii,
               conv_radii,
               layer_type='MCConv',
               sampling_method='poisson disk'):
    super(mymodel, self).__init__()
    self.num_layers = len(sample_radii)
    self.sample_radii = sample_radii.reshape(-1,1)
    self.conv_radii = conv_radii
    self.sampling_method = sampling_method
    self.conv_layers = []
    self.batch_layers = []
    self.dense_layers = []
    self.activations = []
    # encoder
    for i in range(self.num_layers):
      # convolutional downsampling layers
      if layer_type == 'MCConv':
        self.conv_layers.append(layers.MCConv(
            num_features_in=feature_sizes[i],
            num_features_out=feature_sizes[i + 1],
            num_dims=3,
            num_mlps=4,
            mlp_size=[8]))
      elif layer_type == 'PointConv':
        self.conv_layers.append(layers.PointConv(
            num_features_in=feature_sizes[i],
            num_features_out=feature_sizes[i + 1],
            num_dims=3,
            size_hidden=32))
      elif layer_type == 'KPConv':
        self.conv_layers.append(layers.KPConv(
            num_features_in=feature_sizes[i],
            num_features_out=feature_sizes[i + 1],
            num_dims=3,
            num_kernel_points=15))
      else:
        raise ValueError("Unknown layer type!")
      if i < self.num_layers-1:
        # batch normalization and activation function
        self.batch_layers.append(tf.keras.layers.BatchNormalization())
        self.activations.append(tf.keras.layers.LeakyReLU())
    # global pooling
    self.global_pooling = layers.GlobalAveragePooling()
    self.batch_layers.append(tf.keras.layers.BatchNormalization())
    self.activations.append(tf.keras.layers.LeakyReLU())
    # MLP
    self.dense_layers.append(tf.keras.layers.Dense(feature_sizes[-2]))
    self.batch_layers.append(tf.keras.layers.BatchNormalization())
    self.activations.append(tf.keras.layers.LeakyReLU())
    self.dense_layers.append(tf.keras.layers.Dense(feature_sizes[-1]))

  def __call__(self,
               points,
               features,
               training):
    ''' Evaluates network.

    Args:
      points: The point coordinates. Shape `[B, N, 3]`.
      features: Input features. Shape `[B, N, C]`.
      training: A `bool`, passed to the batch norm layers.

    Returns:
      The logits per class.
    '''
    sample_radii = self.sample_radii
    conv_radii = self.conv_radii
    sampling_method = self.sampling_method
    # input point cloud
    # Note: Here all point clouds have the same number of points, so no `sizes`
    #       or `batch_ids` are passed.
    point_cloud = pc.PointCloud(points)
    # spatial downsampling
    point_hierarchy = pc.PointHierarchy(point_cloud,
                                        sample_radii,
                                        sampling_method)
    # network evaluation
    for i in range(self.num_layers):
      features = self.conv_layers[i](features,
                                     point_hierarchy[i], 
                                     point_hierarchy[i+1],
                                     conv_radii[i])
      if i < self.num_layers-1:
        features = self.batch_layers[i](features, training=training)
        features = self.activations[i](features)
    # classification head
    features = self.global_pooling(features, point_hierarchy[-1])
    features = self.batch_layers[-2](features, training)
    features = self.activations[-2](features)
    features = self.dense_layers[-2](features)
    features = self.batch_layers[-1](features, training)
    features = self.activations[-1](features)
    return self.dense_layers[-1](features)


### Model parameters

In [ ]:
batch_size = 16

feature_sizes = [1, 128, 256, 512, 128, num_classes]
sample_radii = np.array([0.1, 0.2, 0.4])
conv_radii = sample_radii * 1.5

# initialize data generators
gen_train = modelnet_data_generator(train_data_points, train_labels, batch_size)
gen_test = modelnet_data_generator(test_data_points, test_labels, batch_size)

# loss function and optimizer
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

lr_decay=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01,
    decay_steps=len(gen_train),
    decay_rate=0.95)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_decay)

### Training Loop

In [ ]:
def training(model,
             optimizer,
             loss_function,
             num_epochs = 10,
             epochs_print=1):
  train_loss_results = []
  train_accuracy_results = []
  test_loss_results = []
  test_accuracy_results = []

  for epoch in range(num_epochs):
    time_epoch_start = time.time()

    # --- Training ---
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    for points, features, labels in gen_train:
      # evaluate model; forward pass
      with tf.GradientTape() as tape:
        logits = model(points, features, training=True)
        pred = tf.nn.softmax(logits, axis=-1)
        loss = loss_function(y_true=labels, y_pred=pred)
      # backpropagation
      grads = tape.gradient(loss, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

      epoch_loss_avg.update_state(loss)
      epoch_accuracy.update_state(labels, pred)

    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    # --- Validation ---
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    for points, features, labels in gen_test:
      # evaluate model; forward pass
      logits = model(points, features, training=False)
      pred = tf.nn.softmax(logits, axis=-1)
      loss = loss_function(y_true=labels, y_pred=pred)

      epoch_loss_avg.update_state(loss)
      epoch_accuracy.update_state(labels, pred)

    test_loss_results.append(epoch_loss_avg.result())
    test_accuracy_results.append(epoch_accuracy.result())

    time_epoch_end = time.time()

    if epoch % epochs_print == 0:
      # End epoch
      print('Epoch {:03d} Time: {:.3f}s'.format(
          epoch,
          time_epoch_end - time_epoch_start))
      print('Training:   Loss: {:.3f}, Accuracy: {:.3%}'.format(
          train_loss_results[-1],
          train_accuracy_results[-1]))
      print('Validation: Loss: {:.3f}, Accuracy: {:.3%}'.format(
          test_loss_results[-1],
          test_accuracy_results[-1]))

#### Train with [Monte-Carlo convolutions](https://www.uni-ulm.de/fileadmin/website_uni_ulm/iui.inst.100/institut/Papers/viscom/2018/hermosilla2018montecarlo.pdf).

In [ ]:
model_MC = mymodel(feature_sizes, sample_radii, conv_radii,
                   layer_type='MCConv')
training(model_MC, optimizer, loss_function)

#### Train with [Kernel Point convolutions](https://arxiv.org/pdf/1904.08889.pdf).

To use the cell average sampling used in the paper, we can simply change the sampling method, which is passed to the point hierarchy constructor.

In [ ]:
model_KP = mymodel(feature_sizes, sample_radii, conv_radii,
                   layer_type='KPConv', sampling_method='cell average')
training(model_KP, optimizer, loss_function)

#### Train with [PointConv convolutions](https://openaccess.thecvf.com/content_CVPR_2019/papers/Wu_PointConv_Deep_Convolutional_Networks_on_3D_Point_Clouds_CVPR_2019_paper.pdf).

In [ ]:
model_PC = mymodel(feature_sizes, sample_radii, conv_radii,
                   layer_type='PointConv')
training(model_PC, optimizer, loss_function)